In [2]:
import numpy as np

import sys
sys.path.insert(0,'..')

import souploader

import pandas as pd

In [32]:
from collections import defaultdict

In [82]:
import srs

In [5]:
num_games = 62
missing = {
'jt': [11,12,13,14,15,43,62],
'jb': [21,22,34,47,57,59,60],
'ms': [8] + list(range(19,37)) + [58],
'kw': list(range(1,12)) + [16,20,25,28,32,40,43,46,52,58,60,62],
'rw': [2,10,11,12,13,24,25,48] + list(range(56,63)),
}

In [6]:
all_games = np.full(num_games, 5)

for i in missing.values():
    for j in i:
        game = j-1
        
        all_games[game] -= 1
all_games

array([4, 3, 4, 4, 4, 4, 4, 3, 4, 3, 2, 3, 3, 4, 4, 4, 5, 5, 4, 3, 3, 3,
       4, 3, 2, 4, 4, 3, 4, 4, 4, 3, 4, 3, 4, 4, 5, 5, 5, 4, 5, 5, 3, 5,
       5, 4, 4, 4, 5, 5, 5, 4, 5, 5, 5, 4, 3, 2, 3, 2, 4, 2])

# load game results

In [87]:
# Load all games
#
# Note the months list!!
#
games_df = srs.load_games(2021, ['december','january','february','march','april'])

In [92]:
# Load just boston games
soup = souploader.load_soup('https://www.basketball-reference.com/teams/BOS/2021_games.html')

table = souploader.soup_get_table_as_pandas(soup, id='games')

table = table[table['Opponent'] != 'Opponent']

table = table.rename(columns={'Unnamed: 7': 'Result'})

table['G'] = pd.to_numeric(table['G'])

table['Tm'] = pd.to_numeric(table['Tm'])
table['Opp'] = pd.to_numeric(table['Opp'])

In [69]:
results = np.empty(len(table))
differential = np.empty(len(table))
for i,j in table.iterrows():
    game = int(j['G'])-1
    results[game] = 1 if j['Result'] == 'W' else 0
    differential[game] = j['Tm'] - j['Opp']

In [75]:
win_by_start = np.zeros(6)
games_by_start = np.zeros(6)
diff_by_start = np.zeros(6)

for idx, (nstarters, win, diff) in enumerate(zip(all_games, results, differential), start=1):
    win_by_start[nstarters] += win
    games_by_start[nstarters] += 1
    diff_by_start[nstarters] += diff
    if nstarters == 2 and win > 0:
        print(table.loc[table['G'] == idx,['G','Date','Opponent','Result','Tm','Opp']])

# win_by_start[4] += win_by_start[5]
# games_by_start[4] += games_by_start[5]
# diff_by_start[4] += diff_by_start[5]

     G               Date       Opponent Result     Tm   Opp
10  11  Fri, Jan 15, 2021  Orlando Magic      W  124.0  97.0


In [76]:
for i, (w,g,d) in enumerate( zip(win_by_start, games_by_start, diff_by_start) ):
    l = g-w
    if g > 0:
        print('{}: {}-{} {:.03f} {:.01f}'.format(i,int(w),int(l),w/g,d/g))

2: 1-4 0.200 1.2
3: 7-8 0.467 -3.4
4: 16-11 0.593 2.5
5: 8-7 0.533 5.1


In [65]:
'{:.01f}'.format(.571*82)

'46.8'

# SRS

In [104]:
missing = table[:len(all_games)][all_games <= 3]['G'].values

In [105]:
hosp_games_df = srs.split_team(games_df, 'Boston Celtics', missing, 'Hospital Celtics')

In [113]:
a = srs.get_srs(hosp_games_df)
a['rank'] = np.arange(len(a))+1

In [114]:
b = srs.get_srs(games_df)
b['rank'] = np.arange(len(b))+1

In [115]:
srs_df = a.join(b, how='outer', lsuffix='_hospital').sort_values(by='srs_hospital',ascending=False)

In [116]:
srs_df

,srs_hospital,rank_hospital,srs,rank
Utah Jazz,8.959173,1,8.890180,1.0
Milwaukee Bucks,6.551915,2,6.459985,2.0
Los Angeles Clippers,6.238262,3,6.257541,3.0
Phoenix Suns,5.511396,4,5.624175,4.0
Denver Nuggets,5.019958,5,4.953344,5.0
Philadelphia 76ers,4.578293,6,4.577433,6.0
Brooklyn Nets,4.080542,7,4.174935,7.0
Boston Celtics,3.683743,8,1.837519,11.0
Los Angeles Lakers,3.376560,9,3.307664,8.0
Dallas Mavericks,2.434808,10,2.365443,9.0
